In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import keras
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer

import matplotlib.pyplot as plt

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.layers import SimpleRNN
from keras import optimizers
from keras.utils.vis_utils import plot_model

import time
from numba import njit, jit, vectorize
from numba import prange

Using TensorFlow backend.


In [3]:
def bigrams(words):
    bigrams = []
    for b in words:
        bigrams.append([b[i:i+2] for i in range(len(b)-1)])
    return bigrams

In [4]:
df = pd.read_csv('./data/dataset.csv')
# print('data shape: {}'.format(df.shape))

X = df['NAME']
y = df['NATIONALITY']

classes = y.unique()
# print(classes)
num_classes = len(y.unique())

# print('number of classes: {}'.format(num_classes))

X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X, y, test_size=0.2, random_state=69)

# print('train data shape X, y: {},{}'.format(X_train_df.shape, y_train_df.shape))
# print('test data shape X, y: {},{}'.format(X_test_df.shape, y_test_df.shape))

In [5]:
X_tokenizer = Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', 
          lower=False, char_level=True, oov_token=None)

y_tokenizer = Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', 
          lower=True, char_level=False, oov_token=None)

In [6]:
X_train = X_train_df.values.astype(str) # Otherwise, there's an error when calling 'fit_on_texts' >> AttributeError: 'int' object has no attribute 'lower'
X_test = X_test_df.values.astype(str) # Otherwise, there's an error when calling 'fit_on_texts' >> AttributeError: 'int' object has no attribute 'lower'

# X_train = bigrams(X_train)

X_tokenizer.fit_on_texts(X_train)
X_train = X_tokenizer.texts_to_sequences(X_train)
X_test = X_tokenizer.texts_to_sequences(X_test)
# print(len(X_tokenizer.index_word))

X_train = X_tokenizer.sequences_to_matrix(X_train, mode='tfidf')
X_test = X_tokenizer.sequences_to_matrix(X_test, mode='tfidf')


In [7]:
# encode from string labels to numerical labels 
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train_df.values.astype(str)) # error without astype(str)
y_test = label_encoder.transform(y_test_df.values.astype(str))
# print(encoder.classes_.shape)
# encoder.inverse_transform(y_train)

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# X_tokenizer.word_counts
# print(len(X_tokenizer.word_counts))

In [8]:
max_features = len(X_tokenizer.word_counts)
# print('max_features = {}'.format(max_features))
batch_size = 23
maxlen = 30
embedding_dims = 50
epochs=20

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, padding="post", maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, padding="post", maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

print('Hyperparameters')
print('max_features = {}'.format(max_features))
print('batch_size = {}'.format(batch_size))
print('maxlen = {}'.format(maxlen))
print('embedding_dims = {}'.format(embedding_dims))
print('epochs = {}'.format(epochs))

def train_model():
    print('Build model...')
    model = Sequential()
    model.add(Embedding(max_features,
                        embedding_dims,
                        input_length=maxlen))
    model.add(SimpleRNN(embedding_dims))

#     model.add(LSTM(maxlen))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    print(model.summary())
    
    print('Train...')

    history = model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
                verbose=0
             )
    score, acc = model.evaluate(X_test, y_test,
                                batch_size=batch_size,
                               verbose=1)


    print('Test score:', score)
    print('Test accuracy:', acc)
#     return model

5700 train sequences
1425 test sequences
Pad sequences (samples x time)
X_train shape: (5700, 30)
X_test shape: (1425, 30)
Hyperparameters
max_features = 54
batch_size = 23
maxlen = 30
embedding_dims = 50
epochs = 20


In [9]:
%time _ = train_model()

Build model...
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 50)            2700      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 50)                5050      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 153       
Total params: 7,903
Trainable params: 7,903
Non-trainable params: 0
_________________________________________________________________
None
Train...


/Users/goople/workspace/slu/spring2020/hpc/ml-project/env/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1425/1425 [==============================] - 0s 79us/step
Test score: 0.9729887019960504
Test accuracy: 0.4863157868385315
CPU times: user 1min 1s, sys: 16.7 s, total: 1min 18s
Wall time: 32.1 s


In [11]:
jitted_train_model = jit(nogil=True)(train_model)
%time _ = jitted_train_model();

<ipython-input-8-92c7b4529182>:24: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "train_model" failed type inference due to: Untyped global name 'Sequential': cannot determine Numba type of <class 'type'>

File "<ipython-input-8-92c7b4529182>", line 26:
def train_model():
    <source elided>
    print('Build model...')
    model = Sequential()
    ^

  def train_model():
/Users/goople/workspace/slu/spring2020/hpc/ml-project/env/lib/python3.7/site-packages/numba/core/object_mode_passes.py:178: NumbaWarning: Function "train_model" was compiled in object mode without forceobj=True.

File "<ipython-input-8-92c7b4529182>", line 24:

def train_model():
^

  state.func_ir.loc))
/Users/goople/workspace/slu/spring2020/hpc/ml-project/env/lib/python3.7/site-packages/numba/core/object_mode_passes.py:188: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecat

Build model...
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 50)            2700      
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 153       
Total params: 7,903
Trainable params: 7,903
Non-trainable params: 0
_________________________________________________________________
None
Train...
1425/1425 [==============================] - 0s 77us/step
Test score: 0.9599297831769575
Test accuracy: 0.48561403155326843
CPU times: user 1min 1s, sys: 16.5 s, total: 1min 18s
Wall time: 32 s


In [12]:
%time _ = jitted_train_model();

Build model...
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 30, 50)            2700      
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 153       
Total params: 7,903
Trainable params: 7,903
Non-trainable params: 0
_________________________________________________________________
None
Train...
1425/1425 [==============================] - 0s 75us/step
Test score: 0.9643213196386371
Test accuracy: 0.4863157868385315
CPU times: user 1min 2s, sys: 17 s, total: 1min 19s
Wall time: 32.7 s


In [14]:
from numba import prange

@jit(parallel=True)
def simulate():
    for _ in prange(3):
        train_model()

%time simulate()

<ipython-input-14-231415e0a5d3>:3: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "simulate" failed type inference due to: Untyped global name 'train_model': cannot determine Numba type of <class 'function'>

File "<ipython-input-14-231415e0a5d3>", line 6:
def simulate():
    <source elided>
    for _ in prange(3):
        train_model()
        ^

  @jit(parallel=True)
<ipython-input-14-231415e0a5d3>:3: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "simulate" failed type inference due to: cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "<ipython-input-14-231415e0a5d3>", line 5:
def simulate():
    for _ in prange(3):
    ^

  @jit(parallel=True)
/Users/goople/workspace/slu/spring2020/hpc/ml-project/env/lib/python3.7/site-packages/numba/core/object_mode_passes.py:178: NumbaWarning: Function "simulate" was compiled in object mode without forceobj=T

Build model...
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 30, 50)            2700      
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 50)                5050      
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 153       
Total params: 7,903
Trainable params: 7,903
Non-trainable params: 0
_________________________________________________________________
None
Train...


/Users/goople/workspace/slu/spring2020/hpc/ml-project/env/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1425/1425 [==============================] - 0s 74us/step
Test score: 0.9934027745849208
Test accuracy: 0.43929824233055115
Build model...
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 30, 50)            2700      
_________________________________________________________________
simple_rnn_7 (SimpleRNN)     (None, 50)                5050      
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 153       
Total params: 7,903
Trainable params: 7,903
Non-trainable params: 0
_________________________________________________________________
None
Train...
1425/1425 [==============================] - 0s 63us/step
Test score: 0.9698440140590333
Test accuracy: 0.48491227626800537
Build model...
Model: "sequential_8"
_________________________________________________________________
La